In [14]:
from langchain_ollama import ChatOllama
llm = ChatOllama(model="llama3.1:latest",temperature=0)

In [15]:
from typing_extensions import TypedDict, Annotated
import operator
class State(TypedDict):
    question:str
    answer:str
    context:Annotated[list, operator.add]


In [16]:
from langchain_community.document_loaders import WikipediaLoader
from langchain_core.messages import HumanMessage, SystemMessage

def search_wikipedia(state):
    
    """ Retrieve docs from wikipedia """

    # Search
    search_docs = WikipediaLoader(query=state['question'], 
                                  load_max_docs=2).load()

     # Format
    formatted_search_docs = "\n\n---\n\n".join(
        [
            f'<Document source="{doc.metadata["source"]}" page="{doc.metadata.get("page", "")}">\n{doc.page_content}\n</Document>'
            for doc in search_docs
        ]
    )

    return {"context": [formatted_search_docs]} 

In [17]:
def generate_answer(state:State):
    """ Node to answer a question """
    question = state["question"]
    context = state["context"]
    answer_template = """Answer the question {question} using this context: {context}"""
    answer_instructions = answer_template.format(question=question, 
                                                       context=context)   
    answer = llm.invoke([SystemMessage(content=answer_instructions)]+[HumanMessage(content="Answer this question")])
    return {"answer":answer}

In [ ]:
from langgraph.graph import StateGraph, START, END

builder = StateGraph(State)

builder.add_node("search_wikipedia", search_wikipedia)
builder.add_node("generate_answer", generate_answer)

builder.add_edge(START, "search_wikipedia")
builder.add_edge("search_wikipedia","generate_answer")
builder.add_edge("generate_answer",END)


graph = builder.compile()

result = graph.invoke({"question": "Who is Ram mohan rao "})
result['answer'].content

"I don't see Gorle Sudhakar mentioned anywhere in the provided text. Can you please provide more context or information about who Gorle Sudhakar is? I'll do my best to help."